<a href="https://colab.research.google.com/github/gp201/RNN--Text-Generation/blob/master/Text_Gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tensorflow-gpu==2.0.0-beta0
import tensorflow as tf

import numpy as np
import os
import time

In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [0]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [0]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [0]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [0]:

print ('{} characters mapped to int-> {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' characters mapped to int-> [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [0]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data - designed to work with infinite sequences,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_2 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [0]:
sampled_indices

array([30, 32,  8, 48, 45, 19, 32, 34, 54, 52,  1, 24, 35, 39, 22, 63,  6,
       21, 56, 17,  2, 26, 32, 12, 55, 37, 32, 51, 27, 22, 34, 60, 28, 52,
       40, 56, 50,  3,  9, 13,  6, 34, 60, 64, 41, 36, 46, 30,  8,  1, 10,
       35, 10,  5, 31, 55, 62,  6, 33, 25,  7, 33, 49, 34, 54,  6, 41, 50,
       17,  3, 33, 29, 31, 63, 15, 11, 59, 43,  9, 63,  6, 18, 62, 13, 12,
       22, 14, 58, 28, 16, 16, 61, 28, 44, 11, 48, 29, 20, 15, 36])

In [0]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "cine of our house, how shall we do?\nWe are not furnish'd like Bohemia's son,\nNor shall appear in Sic"

Next Char Predictions: 
 "RT.jgGTVpn LWaJy,IrE!NT?qYTmOJVvPnbrl$3A,VvzcXhR. :W:'Sqx,UM-UkVp,clE$UQSyC;ue3y,FxA?JBtPDDwPf;jQHCX"


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1738434


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=10

In [0]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10


W0611 10:23:54.621961 140546497996672 util.py:244] Unresolved object in checkpoint: (root).optimizer
W0611 10:23:54.623377 140546497996672 util.py:244] Unresolved object in checkpoint: (root).optimizer.iter
W0611 10:23:54.624482 140546497996672 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_1
W0611 10:23:54.625457 140546497996672 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_2
W0611 10:23:54.626286 140546497996672 util.py:244] Unresolved object in checkpoint: (root).optimizer.decay
W0611 10:23:54.627233 140546497996672 util.py:244] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0611 10:23:54.630182 140546497996672 util.py:244] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.embeddings
W0611 10:23:54.631188 140546497996672 util.py:244] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-2.kernel
W0611 10:23:54.632994 140546497996672 util.py:2

172/172 [==============================] - 49s 283ms/step - loss: 2.5630
Epoch 2/10
172/172 [==============================] - 47s 275ms/step - loss: 1.8727
Epoch 3/10
172/172 [==============================] - 48s 277ms/step - loss: 1.6269
Epoch 4/10
172/172 [==============================] - 47s 276ms/step - loss: 1.4932
Epoch 5/10
172/172 [==============================] - 47s 275ms/step - loss: 1.4094
Epoch 6/10
172/172 [==============================] - 47s 275ms/step - loss: 1.3480
Epoch 7/10
172/172 [==============================] - 47s 275ms/step - loss: 1.2965
Epoch 8/10
172/172 [==============================] - 47s 275ms/step - loss: 1.2490
Epoch 9/10
172/172 [==============================] - 47s 274ms/step - loss: 1.2032
Epoch 10/10
172/172 [==============================] - 48s 277ms/step - loss: 1.1599


In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
def generate_text(model, start_string):

  # Number of characters to generate
  num_generate = 1000

  # vectorizing
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: nghe word:
That should the house to benity your
will does fiel or I will suffer; my meaning knee,
What crow thy sakes, with the reating other prince, I
have nothing bless you?

ANGELO:
He is not so, his sarfentue fiends, can our embures
Of night, honourable to engloss now.

HASTINIS:
This is the cause;
For my schapless special, in: ours all:
Neither my youth or barrent him to him.
But, sirs, I find.

ROMEO:
Fill in a dream cannocy Boland Edward's
Richard hithess remission. Where we can.
pait we person' ceme to man.
What wouldst thou by, and do you pluck simple:
Tuinous, sister? here, my master has man.

DUCHESS OF YORK:
Withar, care. Camillo, by my brother's death?

SLY:
Ay, as son ono.
Though hand:
He will assurance, nurses you to speak. Marcius,
That, had am love; though lost warm Tybalt deed,
And him unlike thee?

DORCAS:
At our commonwer.

MENENIUS:
Comitly, or; bell me thou wilt be splant,
Death as an ensolution.

DUCHESS OF YORK:
Nay, that sacret friend, the boaths since y